In [1]:
# """Install required Python packages"""
# !pip install mmi==0.1.19
# !pip install bokeh==0.12.7
# !pip install ipywidgets==5.2.2
# !pip install matplotlib==1.5.1

In [2]:
"""Import required packages"""
import mmi.mmi_client
import mmi.tracker as tr
import numpy as np

from bokeh.io import push_notebook, output_notebook, curdoc
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.models.widgets import Slider
from bokeh.layouts import widgetbox, Column, row
 
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual

import threading
import time

In [3]:
"""Make connection with running model in docker container"""
model = mmi.mmi_client.MMIClient('tcp://xbeach-run:53606')

In [4]:
"""Initialize arrays"""
H, zs, zb, water, times = [], [], [], [], [0]
zsarr = model.get_var('zs')[0]
zbarr = model.get_var('zb')[0]
Harr = model.get_var('H')[0]
H.append(Harr)
zs.append(zsarr)
zb.append(zbarr)
water.append(Harr + zsarr)

In [5]:
"""Create two functions. update_plot for updating when slider changes. 
    update for interaction with the model"""

def update_plot(time):
    try:
        print(times[time])
        p.title.text = ('timestep: ' + str(times[time]))
        waterline.data_source.data['y'] = water[time]
    except:
        p.title.text = ('timestep: ' + str(times[time]) + ', does not exist yet')
    push_notebook()
    
slider = interactive(update_plot, time=(0, len(times), 1 ))

def update():
    t_old = model.get_current_time()
    while model.get_current_time() <= model.get_end_time():
        t_new = model.get_current_time()
        if (t_old != t_new):
            times.append(t_old)
            slider.children[0].max = len(times) - 1
            zsarr = model.get_var('zs')[0]
            zbarr = model.get_var('zb')[0]
            Harr = model.get_var('H')[0]
            H.append(Harr)
            zb.append(zbarr)
            zs.append(zsarr)
            water.append(Harr + zsarr)
        t_old = t_new
    time.sleep(1)

In [6]:
"""Build plot in bokeh"""
output_notebook()
tstep = int(model.get_current_time())

TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select,lasso_select"
p = figure(tools=TOOLS, width=700, height=350, title='timestep: '+ str(tstep), 
          x_axis_label='cell in x-direction', y_axis_label='height [m]')

bath = p.line(np.arange(len(zb[0])), zb[0], line_width=2, color='black')
waterline = p.line(np.arange(len(zb[0])), water[0], color='blue')

hover = HoverTool(renderers=[waterline])

hover.tooltips = [
    ("Water level", "$y")
]
p.add_tools(hover)

show(p, notebook_handle=True)

Loading BokehJS ...

In [7]:
"""Create connection with """
thread = threading.Thread(target=update)
display(slider)
thread.start()

24615.789473684206
